In [1]:
## imports

import re
import random
import datetime
import glob
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
pd.options.mode.chained_assignment = None 

In [2]:
## data source: masseyratings.com

In [3]:
## rankings that are used during the 2022 season

rankers = pd.read_csv('cf2022.csv')
rankers = list(rankers.ranking_code.unique())

In [4]:
## cleaning txt files, used to document each season's games

team_code = {}
team_number = 500

def clean_data(data):
    global team_code
    global team_number
    games = []
    seed = 3
    
    for line in data:
        dates = re.findall('[0-9]{4}-[0-9]{2}-[0-9]{2}', line) ## regex for dates
        line = re.sub(r'[0-9]{4}-[0-9]{2}-[0-9]{2}', r'', line) ## removing dates from line
        date = dates[0]
        line = line.strip()

        teams = re.findall("['@A-Za-z\s&]+?(?=[0-9])", line) ## regex for 2 teams
        
        random.seed(seed)
        if len(teams) > 1: ## if statement for lines that aren't pulling back anything
            r = random.uniform(0, 1)
            if r < 0.5:
                team1 = teams[0].strip()
                team2 = teams[1].strip()
            else:
                team1 = teams[1].strip()
                team2 = teams[0].strip()
        seed += 1
        line = re.sub(r"['@A-Za-z\s&]+?(?=[0-9])", r' ', line) ## removing teams from line
        
        ## variables that are available for use but are removed in current model
        ## score differential, home team
        scores = re.findall('[0-9]{1,3}', line) ## regex for 2 scores
        if len(scores) > 1: ## if statement for lines that aren't pulling back anything
            if r < 0.5:
                score1 = int(scores[0].strip())
                score2 = int(scores[1].strip())
            else:
                score1 = int(scores[1].strip())
                score2 = int(scores[0].strip())
            score_diff = abs(score1 - score2) ## difference in scores between the two teams
            if score_diff == 0:
                score1 = np.nan
                score2 = np.nan
                score_diff = np.nan
        
        team1_host = re.findall('@', team1)
        team1 = re.sub('@', '', team1)

        team2_host = re.findall('@', team2)
        team2 = re.sub('@', '', team2)
        
        ## team codes
        if team1 not in team_code:
            team_code[team1] = team_number
            team_number = team_number + 1
        if team2 not in team_code:
            team_code[team2] = team_number ## assigning number to team
            team_number = team_number + 1 ## adding 1 to number assignment, avoiding dup assignments
         
        if len(team1_host) > 0:
            host_team = team_code[team1] ## who is hosting the game? assigning team number to variable
        elif len(team2_host) > 0:
            host_team = team_code[team2] ## who is hosting the game? assigning team number to variable
        else:
            host_team = 0 ## how to handle neutral site?
            
        ## who won?
        if score1 > score2:
            winner = 'team1'
        elif score2 > score1:
            winner = 'team2'
        else:
            winner = 'tbd' ## game has not happened yet


        games.append((date, team1, team2, score1, score2, score_diff, host_team, winner))

    games = pd.DataFrame(games)
    games.columns = ['date', 'team1', 'team2', 'score1', 'score2', 'score_diff', 'host_team', 'winner']
    games['team1_code'] = games["team1"].apply(lambda x: team_code.get(x))
    games['team2_code'] = games["team2"].apply(lambda x: team_code.get(x))
    games = games[['team1', 'team2', 'winner', 'team1_code', 'team2_code']]
    
    games = games.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    
    return games

In [5]:
## all files but 2022 need cleaning

def clean_rankings(rankings):
    rankings = rankings.rename(columns={0: 'year', 1: 'team_code', 2: 'team_name', 3: 'ranking_code',
                                        4: 'ranking_name', 5: 'date_str', 6: 'rank'})
    rankings['date'] = pd.to_datetime(rankings['date_str'], format='%Y%m%d')
    rankings = rankings[rankings['date'] == rankings['date'].max()]
    rankings['week'] = 1
    rankings = rankings[['year', 'team_name', 'ranking_name', 'ranking_code', 'rank', 'week']]
    rankings = rankings.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    rankings = rankings[rankings['ranking_code'].isin(rankers)]
    
    return rankings

In [6]:
## formatting to compare rankings on head-to-head matchups

def set_rankings(rankings, games):

    rankers = rankings.ranking_code.unique()

    rankers_adj = []

    for ranker in rankers:
        ranker = ranker + '_team_1'
        rankers_adj.append(ranker)
    for ranker in rankers:
        ranker = ranker + '_team_2'
        rankers_adj.append(ranker)


    for ranker in rankers_adj:
        games[ranker] = np.nan

    games = games.set_index('team2')
    for index, row in rankings.iterrows():
        games[row[3] + '_team_2'][row[1]] = row[4]

    games = games.reset_index()
    games = games.set_index('team1')
    for index, row in rankings.iterrows():
        games[row[3] + '_team_1'][row[1]] = row[4]

    games = games.reset_index()
    for i in games:
        if i != 'winner':
            games[i] = games[i].fillna(350)

    return games

In [7]:
## building final document

def compile_rankings():
    
    rankings = pd.DataFrame()

    for file in os.listdir():
        if '.txt' in file:
            year = file[2:6]
            with open(file) as f:
                lines = f.readlines()
            games = clean_data(lines)
            if year != '2022' and rankings.empty:
                rankings_temp = pd.read_csv('cf' + year + '.csv', header = None)
                rankings_temp = clean_rankings(rankings_temp)
                rankings_temp = set_rankings(rankings_temp, games)
                rankings = rankings_temp
            elif year != '2022':
                rankings_temp = pd.read_csv('cf' + year + '.csv', header = None)
                rankings_temp = clean_rankings(rankings_temp)
                rankings_temp = set_rankings(rankings_temp, games)
                rankings = pd.concat([rankings, rankings_temp], ignore_index = True)
            else:
                rankings_temp = pd.read_csv('cf' + year + '.csv')
                rankings_temp = set_rankings(rankings_temp, games)
                rankings = pd.concat([rankings, rankings_temp], ignore_index = True)
    
    rankings = rankings.dropna(axis = 'columns')
    return rankings

In [8]:
rankings = compile_rankings()

In [9]:
rankings

,team1,team2,winner,team1_code,team2_code,AND_team_1,COL_team_1,ARG_team_1,MAS_team_1,AP_team_1,...,FEI_team_2,HEN_team_2,STH_team_2,PGH_team_2,CTW_team_2,SFX_team_2,PPP_team_2,PIR_team_2,TPR_team_2,DII_team_2
0,California,Hawaii,team1,500,501,62.0,66.0,63.0,62.0,350.0,...,111.0,85.0,92.0,82.0,97.0,109.0,84.0,91.0,111.0,94.0
1,N Dakota St,Charleston So,team1,502,503,350.0,350.0,350.0,2.0,350.0,...,350.0,350.0,14.0,20.0,15.0,350.0,350.0,350.0,350.0,13.0
2,Presbyterian,C Michigan,team2,504,505,350.0,350.0,350.0,97.0,350.0,...,104.0,93.0,98.0,96.0,95.0,102.0,93.0,93.0,84.0,89.0
3,TN Martin,Cincinnati,team2,506,507,350.0,350.0,350.0,40.0,350.0,...,103.0,96.0,101.0,100.0,92.0,90.0,95.0,99.0,87.0,95.0
4,Connecticut,Maine,team1,508,509,112.0,117.0,125.0,117.0,350.0,...,350.0,350.0,28.0,25.0,39.0,350.0,350.0,350.0,350.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14972,Kentucky,Penn St,team1,600,615,10.0,13.0,13.0,11.0,12.0,...,17.0,14.0,12.0,20.0,14.0,9.0,17.0,19.0,20.0,22.0
14973,Washington,Ohio St,team2,691,613,18.0,18.0,16.0,19.0,13.0,...,6.0,4.0,7.0,4.0,3.0,4.0,4.0,3.0,6.0,6.0
14974,Georgia,Texas,team2,610,754,6.0,6.0,4.0,5.0,7.0,...,16.0,18.0,22.0,14.0,19.0,9.0,18.0,16.0,13.0,14.0
14975,E Washington,N Dakota St,team2,693,502,350.0,350.0,350.0,3.0,350.0,...,350.0,350.0,1.0,1.0,1.0,350.0,350.0,350.0,350.0,1.0


In [10]:
X_played = rankings[rankings['winner'] != 'tbd'] ## all games that have been played in data set
X_dev = rankings[rankings['winner'] == 'tbd'] ## pulling out bowl games, predicting these

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_played, X_played['winner'], test_size = 0.2, 
                                                    random_state = 3) ## splitting data

X_train = X_train.drop(columns = ['winner', 'team1', 'team2'])
X_test = X_test.drop(columns = ['winner', 'team1', 'team2'])

In [12]:
X_dev = X_dev.drop(columns = ['winner', 'team1', 'team2'])
X_dev.head()

,team1_code,team2_code,AND_team_1,COL_team_1,ARG_team_1,MAS_team_1,AP_team_1,BAS_team_1,PFZ_team_1,BIH_team_1,...,FEI_team_2,HEN_team_2,STH_team_2,PGH_team_2,CTW_team_2,SFX_team_2,PPP_team_2,PIR_team_2,TPR_team_2,DII_team_2
10904,502,553,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,...,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0
10905,527,514,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,...,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0
10906,740,714,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,...,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0
10907,574,607,94.0,82.0,82.0,91.0,350.0,72.0,67.0,101.0,...,68.0,85.0,79.0,90.0,66.0,70.0,106.0,79.0,74.0,79.0
10908,661,733,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,...,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0


In [13]:
## building + fitting decision tree classifier

winner_model = RandomForestClassifier(criterion = 'log_loss', random_state = 3).fit(X_train, y_train)
score = winner_model.score(X_test, y_test) ## score = percent at which model is predicting correctly
print(score)

## using model for predictions
y_pred = winner_model.predict(X_dev)
X_dev['winner'] = y_pred

d = {v:k for k, v in team_code.items()}
X_dev['team1_code'] = X_dev['team1_code'].map(d)
X_dev['team2_code'] = X_dev['team2_code'].map(d)

first_column = X_dev.pop('winner')
X_dev.insert(0, 'winner', first_column)

0.8131279303415941


In [14]:
X_dev

,winner,team1_code,team2_code,AND_team_1,COL_team_1,ARG_team_1,MAS_team_1,AP_team_1,BAS_team_1,PFZ_team_1,...,FEI_team_2,HEN_team_2,STH_team_2,PGH_team_2,CTW_team_2,SFX_team_2,PPP_team_2,PIR_team_2,TPR_team_2,DII_team_2
10904,team1,N Dakota St,Samford,350.0,350.0,350.0,350.0,350.0,350.0,350.0,...,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0
10905,team1,Montana St,William & Mary,350.0,350.0,350.0,350.0,350.0,350.0,350.0,...,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0
10906,team1,CS Sacramento,Incarnate Word,350.0,350.0,350.0,350.0,350.0,350.0,350.0,...,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0
10907,team2,Army,Navy,94.0,82.0,82.0,91.0,350.0,72.0,67.0,...,68.0,85.0,79.0,90.0,66.0,70.0,106.0,79.0,74.0,79.0
10908,team2,S Dakota St,Holy Cross,350.0,350.0,350.0,350.0,350.0,350.0,350.0,...,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0
10909,team1,UAB,Miami OH,92.0,74.0,72.0,87.0,350.0,79.0,70.0,...,102.0,105.0,115.0,104.0,99.0,97.0,102.0,99.0,104.0,104.0
10910,team2,UT San Antonio,Troy,27.0,20.0,28.0,52.0,22.0,58.0,38.0,...,56.0,30.0,46.0,25.0,24.0,62.0,16.0,67.0,62.0,56.0
10911,team1,Jackson St,NC Central,350.0,350.0,350.0,350.0,350.0,350.0,350.0,...,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0
10912,team2,Cincinnati,Louisville,38.0,31.0,32.0,34.0,350.0,30.0,40.0,...,26.0,31.0,27.0,34.0,27.0,33.0,28.0,28.0,33.0,32.0
10913,team1,SMU,BYU,57.0,48.0,54.0,55.0,350.0,59.0,52.0,...,66.0,62.0,61.0,56.0,36.0,57.0,51.0,55.0,56.0,68.0


In [15]:
## championship game has yet to be determined, pulling winners from the semi-final games

def championship_game():

    X_dev_rankers = []

    for x in X_dev.columns:
        if x == 'winner':
            pass
        elif x[:4] == 'team':
            pass
        elif x[:2] == 'AP':
            x = x[:2]
            if x not in X_dev_rankers:
                X_dev_rankers.append(x)
        else:
            x = x[:3]
            if x not in X_dev_rankers:
                X_dev_rankers.append(x)

    X_dev_chmp = pd.DataFrame(columns = X_dev.columns)
    X_dev_chmp['winner'] = 1
    X_dev_chmp
    X_dev_dict = {'winner': np.nan, 'team1': 'Michigan', 'team2': 'Georgia', 'team1_code': 606, 'team2_code': 610}
    X_dev_dict = pd.DataFrame([X_dev_dict])
    X_dev_chmp = pd.concat([X_dev_chmp, X_dev_dict], ignore_index = True)

    rankings2022 = pd.read_csv('cf2022.csv')
    rankings2022 = rankings2022[rankings2022['ranking_code'].isin(X_dev_rankers)]

    X_dev_chmp = X_dev_chmp.set_index('team2')
    for index, row in rankings2022.iterrows():
        X_dev_chmp[row[3] + '_team_2'][row[1]] = row[4]

    X_dev_chmp = X_dev_chmp.reset_index()
    X_dev_chmp = X_dev_chmp.set_index('team1')
    for index, row in rankings2022.iterrows():
        X_dev_chmp[row[3] + '_team_1'][row[1]] = row[4]


    X_dev_chmp = X_dev_chmp.reset_index()   

    X_dev_chmp = X_dev_chmp.drop(columns = ['winner', 'team1', 'team2'])


    y_pred = winner_model.predict(X_dev_chmp)
    X_dev_chmp['winner'] = y_pred

    d = {v:k for k, v in team_code.items()}
    X_dev_chmp['team1_code'] = X_dev_chmp['team1_code'].map(d)
    X_dev_chmp['team2_code'] = X_dev_chmp['team2_code'].map(d)

    first_column = X_dev_chmp.pop('winner')
    X_dev_chmp.insert(0, 'winner', first_column)

    return X_dev_chmp

In [16]:
championship_game()

,winner,team1_code,team2_code,AND_team_1,COL_team_1,ARG_team_1,MAS_team_1,AP_team_1,BAS_team_1,PFZ_team_1,...,FEI_team_2,HEN_team_2,STH_team_2,PGH_team_2,CTW_team_2,SFX_team_2,PPP_team_2,PIR_team_2,TPR_team_2,DII_team_2
0,team2,Michigan,Georgia,2.0,2.0,2.0,2.0,2.0,3.0,2.0,...,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0
